In [1]:
# The code was removed by Watson Studio for sharing.

# Segmenting and Clustering Neighborhoods in Toronto

## Part 1: Scraping and Cleaning

### Scraping the wikipage

Let's start by importing the necessary libraries.

In [2]:
# Importing libraries
import pandas as pd
import numpy as np
import requests # for making standard html requests
from sklearn.cluster import KMeans

!pip install folium
import folium # map rendering library

# Importing geocoder
!pip install geocoder
import geocoder

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import copy

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

print('Libraries Imported!')

Libraries Imported!


Next up: scraping the page using pd.read_html()

In [3]:
# Request data target URL
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
df = pd.read_html(url, header = 0)

In [4]:
# Assign the table to the variable df
df = df[0]
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


### Data Cleaning

Not interested in Boroughs that are not assigned. Let's remove these from the dataset.

In [5]:
# Borough should not be 'Not assigned'
print('Count "Not assigned" in Borough: {}'.format(list(df['Borough'] == "Not assigned").count(True)))
df = df.iloc[list(df['Borough'] != "Not assigned")].reset_index().drop(['index'], axis = 1)
df.head()

Count "Not assigned" in Borough: 77


,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


Checking if there are any Neighbourhoods with the value 'Not assigned'.

In [6]:
#There are no cells with the value 'Not assigned'
print('Count "Not assigned" in Neighbourhood: {}'.format(list(df['Neighbourhood'] == "Not assigned").count(True)))

Count "Not assigned" in Neighbourhood: 0


Checking for duplicates. If the returned dictionary is empty, there are none found.

In [7]:
def getDuplicatesWithCount(listOfElems):
    ''' Get frequency count of duplicate elements in the given list '''
    dictOfElems = dict()
    # Iterate over each element in list
    for elem in listOfElems:
        # If element exists in dict then increment its value else add it in dict
        if elem in dictOfElems:
            dictOfElems[elem] += 1
        else:
            dictOfElems[elem] = 1    
 
    # Filter key-value pairs in dictionary. Keep pairs whose value is greater than 1 i.e. only duplicate elements from list.
    dictOfElems = { key:value for key, value in dictOfElems.items() if value > 1}
    # Returns a dict of duplicate elements and thier frequency count
    return dictOfElems

In [8]:
# There are no duplicates.
dictOfElems = getDuplicatesWithCount(list(df['Postal Code']))
dictOfElems

{}

Checking shape.

In [9]:
print('Number of rows: {}'.format(df.shape[0]))

Number of rows: 103


## Part 2: assigning geographical coordinates

### Geocoder

Let's try out the Geocoder package and the geocoder.google function. First let's install geocoder

The code performs as follows:

1. Initiliazation of values:
    1. Lists that store our values.
    1. i is to count the number of times we have tried a certain postal code. It is to prevent an infinite loop.
    1. lat_lng_coords temporarily stores the coordinates of a single postal code
    1. g stores the API call
1. A for loop will go over every postal code in the data frame
    1. The initial values will be reset
    1. The address will be formatted
    1. The address will be called in a while loop, using i as a safety
    1. If the API has found the coordinates, they will be appended to their respective lists. If not, None values will be appended.

In [10]:
list_lat = []
list_long = []
i = 0
lat_lng_coords = None
g = None

for postal_code in df['Postal Code']:
    lat_lng_coords = None
    g = None
    i = 0
    address = '{}, Toronto, Ontario'.format(postal_code)
    while(lat_lng_coords is None and i < 5):
        g = geocoder.google(address)
        lat_lng_coords = g.latlng
        i = i + 1
    if lat_lng_coords != None:
        list_lat.append(lat_lng_coords[0])
        list_long.append(lat_lng_coords[1])
    else:
        list_lat.append(None)
        list_long.append(None)
    print(address)

M3A, Toronto, Ontario
M4A, Toronto, Ontario
M5A, Toronto, Ontario
M6A, Toronto, Ontario
M7A, Toronto, Ontario
M9A, Toronto, Ontario
M1B, Toronto, Ontario
M3B, Toronto, Ontario
M4B, Toronto, Ontario
M5B, Toronto, Ontario
M6B, Toronto, Ontario
M9B, Toronto, Ontario
M1C, Toronto, Ontario
M3C, Toronto, Ontario
M4C, Toronto, Ontario
M5C, Toronto, Ontario
M6C, Toronto, Ontario
M9C, Toronto, Ontario
M1E, Toronto, Ontario
M4E, Toronto, Ontario
M5E, Toronto, Ontario
M6E, Toronto, Ontario
M1G, Toronto, Ontario
M4G, Toronto, Ontario
M5G, Toronto, Ontario
M6G, Toronto, Ontario
M1H, Toronto, Ontario
M2H, Toronto, Ontario
M3H, Toronto, Ontario
M4H, Toronto, Ontario
M5H, Toronto, Ontario
M6H, Toronto, Ontario
M1J, Toronto, Ontario
M2J, Toronto, Ontario
M3J, Toronto, Ontario
M4J, Toronto, Ontario
M5J, Toronto, Ontario
M6J, Toronto, Ontario
M1K, Toronto, Ontario
M2K, Toronto, Ontario
M3K, Toronto, Ontario
M4K, Toronto, Ontario
M5K, Toronto, Ontario
M6K, Toronto, Ontario
M1L, Toronto, Ontario
M2L, Toron

In [11]:
print(list_lat)
print(list_long)

[None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]
[None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, No

Didn't obtain any data either. In that case, let us proceed using the provided data.

### Data Merge using provided data

Since we can't obtain the information with Geocoder either. We will continue using the data provided by the instructor.

In [12]:
# Code if we can't get any coordinates with Geocoder. Project is the project called from a hidden cell using project_lib.
geospatial_coordinates = pd.read_csv(project.get_file("Geospatial_Coordinates.csv"))
geospatial_coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Now that we have acquired the latitude and longitude data. Let's merge the data into a single dataframe.

In [13]:
df_full = df
df_full = df_full.merge(right = geospatial_coordinates, on = 'Postal Code')
df_full.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [14]:
df_full.shape

(103, 5)

## Segmenting and Clustering


In [15]:
# The code was removed by Watson Studio for sharing.

### 1. Requesting data through Foursquare API calls

Through the Foursquare API calls we will request the necessary data for clustering. All neighborhoods will be used, not only the Boroughs that contain Toronto.

In [16]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Now that we have defined the function to call the necessary data, let's use it to call the necessary Toronto data.

In [18]:
toronto_venues = getNearbyVenues(df_full['Neighbourhood'], df_full['Latitude'], df_full['Longitude'])

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmo

In [19]:
len(toronto_venues['Neighbourhood'].unique())

96

Only 96 neighbourhoods, meaning we didn't receive data for 7 neighbourhoods. The cause could be too small radius that in an absence of venues within the specified radius.

### 2. Exploring the Neighborhood

In [20]:
toronto_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Agincourt,5,5,5,5,5,5
"Alderwood, Long Branch",7,7,7,7,7,7
"Bathurst Manor, Wilson Heights, Downsview North",21,21,21,21,21,21
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",22,22,22,22,22,22
...,...,...,...,...,...,...
"Willowdale, Willowdale West",5,5,5,5,5,5
Woburn,4,4,4,4,4,4
Woodbine Heights,7,7,7,7,7,7


In [21]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 237 uniques categories.


### 3. Analyze each Neighbourhood

Next up is to use One Hot Encoding to build our matrix. This matrix will allow us to apply KMeans Clustering

In [22]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighbourhood,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
#The size of our matrix
toronto_onehot.shape

(1317, 238)

In [24]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

,Neighbourhood,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.045455,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,"Willowdale, Willowdale West",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
92,Woburn,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
93,Woodbine Heights,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
94,York Mills West,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:
toronto_grouped.shape

(96, 238)

In [26]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [27]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Lounge,Latin American Restaurant,Breakfast Spot,Skating Rink,Clothing Store,Dessert Shop,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore
1,"Alderwood, Long Branch",Pizza Place,Sandwich Place,Coffee Shop,Pub,Pharmacy,Gym,General Entertainment,Drugstore,Dog Run,Distribution Center
2,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Pharmacy,Chinese Restaurant,Shopping Mall,Diner,Sandwich Place,Deli / Bodega,Restaurant,Supermarket
3,Bayview Village,Bank,Chinese Restaurant,Japanese Restaurant,Café,Yoga Studio,Dessert Shop,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore
4,"Bedford Park, Lawrence Manor East",Coffee Shop,Italian Restaurant,Sandwich Place,Sushi Restaurant,Greek Restaurant,Indian Restaurant,Juice Bar,Liquor Store,Locksmith,Comfort Food Restaurant


### 4. Clustering Neighbourhoods

The following function is the same code as used in the New York Analysis. This is done in order to visually explore the different possibilities for K without having to copy all the code. In addition, it will also return the merged df so we can use it for later analysis.

Note that we are not applying any validation statistics on our number of clusters. This can result in a non-optimal solution for our chosen number of clusters. Any result gained is thus subjective.

In [28]:
def ClusteringAndPlotting(kclusters, df_grouped, df_full, neighbourhoods_venues_sorted, address = 'Toronto, Ontario'):
    df_grouped_clustering = df_grouped.drop('Neighbourhood', 1)
    # run k-means clustering
    kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(df_grouped_clustering)
    neighbourhoods_venues_insert = copy.deepcopy(neighbourhoods_venues_sorted)
    neighbourhoods_venues_insert.insert(0, 'Cluster Labels', kmeans.labels_)
    df_merged = df_full
    # merge df_grouped with df_full to add latitude/longitude for each neighborhood
    df_merged = df_merged.join(neighbourhoods_venues_insert.set_index('Neighbourhood'), on='Neighbourhood')
    
    # Address
    geolocator = Nominatim(user_agent="foursquare_agent")
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    
    # Drop observations that have a NaN value
    df_merged_na = df_merged.replace([np.nan, -np.nan], np.nan).dropna()
    
    # create map
    map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

    # Set Cluster Labels as integers instead of floats
    df_merged_na['Cluster Labels'] = df_merged_na['Cluster Labels'].astype(int)

    # set color scheme for the clusters
    x = np.arange(kclusters)
    ys = [i + x + (i*x)**2 for i in range(kclusters)]
    colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
    rainbow = [colors.rgb2hex(i) for i in colors_array]

    # add markers to the map
    markers_colors = []
    for lat, lon, poi, cluster in zip(df_merged_na['Latitude'], df_merged_na['Longitude'], df_merged_na['Neighbourhood'], df_merged_na['Cluster Labels']):
        label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
        folium.CircleMarker(
            [lat, lon],
            radius=5,
            popup=label,
            color=rainbow[cluster-1],
            fill=True,
            fill_color=rainbow[cluster-1],
            fill_opacity=0.7).add_to(map_clusters)

    return(df_merged_na, map_clusters)

    

We will start with seven clusters and will decrease or increase depending on the situation. 

For example, clusters containing only a single data point might be an artefact introduced by the algorithm since we force our algorithm to have at least one data point in each cluster.
If there are no single data point clusters, we might not have reached full saturation and our obtained clusters might still contain subclusters.

In [29]:
# kclusters = 7
toronto_merged_7, map_clusters_7 = ClusteringAndPlotting(7, toronto_grouped, df_full, neighbourhoods_venues_sorted)
map_clusters_7

The clustering algorithm identifies only 3 clusters that aren't single data point clusters. Single data point clusters are questionable. Are they truly clusters or are they artefacts introduced by the algorithm?

We will decrease the number of clusters to 4 to evaluate the effects.

In [30]:
# kclusters = 4
toronto_merged_4, map_clusters_4 = ClusteringAndPlotting(4, toronto_grouped, df_full, neighbourhoods_venues_sorted)
map_clusters_4

There is still a single data point cluster. This is probably still an artefact. We will decrease the number of clusters to 3.

In [31]:
# kclusters = 3
toronto_merged_3, map_clusters_3 = ClusteringAndPlotting(3, toronto_grouped, df_full, neighbourhoods_venues_sorted)
map_clusters_3

Our dataset doesn't contain any more single data point clusters. We will have a closer look on the obtained clusters in the next section.

### 5. Examing Clusters

In the following section we will identify the shared Most Common Venues. This is done on sight and is thus quite subjective.

A quick look on the first cluster showes a couple of shared venues in the following order of importance:
1. Parks are the most common venue
1. Yoga Studios are also quite common
1. Department Stores and Electronic Stores
1. Eastern European Restaurant, Dumpling Restaurant
1. Drugstores and Donut Shops

In [32]:
# The First Cluster
toronto_merged_3.loc[toronto_merged_3['Cluster Labels'] == 0, toronto_merged_3.columns[[1] + list(range(5, toronto_merged_3.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,0,Park,Food & Drink Shop,Yoga Studio,Department Store,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Dog Run
21,York,0,Park,Women's Store,Pool,Yoga Studio,Deli / Bodega,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Dog Run
35,East York,0,Park,Convenience Store,Intersection,Yoga Studio,Department Store,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop
49,North York,0,Park,Construction & Landscaping,Bakery,Yoga Studio,Department Store,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop
61,Central Toronto,0,Park,Swim School,Bus Line,Yoga Studio,Department Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Dog Run
64,York,0,Park,Yoga Studio,Deli / Bodega,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center
66,North York,0,Park,Convenience Store,Yoga Studio,Department Store,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Dog Run
68,Central Toronto,0,Jewelry Store,Park,Sushi Restaurant,Trail,Distribution Center,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Yoga Studio
85,Scarborough,0,Playground,Park,Bakery,Yoga Studio,Deli / Bodega,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Dog Run
91,Downtown Toronto,0,Park,Trail,Playground,Yoga Studio,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run


In the next piece of code we will have a look at the second cluster.

Due to the amount of data points in the second cluster, it is not possible to determine the most common venues by sight alone. We will have a look at the numbers in the most common venues generated by our getDuplicatesWithCount function and look at the venues with a high count.

In [33]:
# The Second Cluster
toronto_merged_3.loc[toronto_merged_3['Cluster Labels'] == 1, toronto_merged_3.columns[[1] + list(range(5, toronto_merged_3.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,1,Pizza Place,Intersection,Coffee Shop,Hockey Arena,Portuguese Restaurant,Diner,Department Store,Dessert Shop,Dim Sum Restaurant,Distribution Center
2,Downtown Toronto,1,Coffee Shop,Park,Bakery,Breakfast Spot,Pub,Chocolate Shop,Distribution Center,Spa,Dessert Shop,Farmers Market
3,North York,1,Clothing Store,Accessories Store,Boutique,Gift Shop,Furniture / Home Store,Event Space,Coffee Shop,Women's Store,Vietnamese Restaurant,Airport Terminal
4,Downtown Toronto,1,Coffee Shop,Yoga Studio,Café,Bar,Italian Restaurant,Beer Bar,Sandwich Place,Distribution Center,Diner,Mexican Restaurant
6,Scarborough,1,Fast Food Restaurant,Yoga Studio,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center
...,...,...,...,...,...,...,...,...,...,...,...,...
97,Downtown Toronto,1,Café,Coffee Shop,Restaurant,Seafood Restaurant,Hotel,Bookstore,Speakeasy,Steakhouse,Bakery,Pizza Place
98,Etobicoke,1,Pool,River,Yoga Studio,Dance Studio,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center
99,Downtown Toronto,1,Coffee Shop,Gay Bar,Restaurant,Indian Restaurant,Beer Bar,Ethiopian Restaurant,Escape Room,Juice Bar,Breakfast Spot,Bubble Tea Shop
100,East Toronto,1,Light Rail Station,Auto Workshop,Comic Shop,Pizza Place,Recording Studio,Restaurant,Butcher,Burrito Place,Skate Park,Brewery


In [34]:
# 1st Most Common Venue
getDuplicatesWithCount(toronto_merged_3.loc[toronto_merged_3['Cluster Labels'] == 1, ['1st Most Common Venue']]['1st Most Common Venue'].tolist())

{'Pizza Place': 9,
 'Coffee Shop': 13,
 'Fast Food Restaurant': 2,
 'Gym': 2,
 'Café': 9,
 'Grocery Store': 8,
 'Playground': 2,
 'Breakfast Spot': 2}

In [35]:
# 2nd Most Common Venue
getDuplicatesWithCount(toronto_merged_3.loc[toronto_merged_3['Cluster Labels'] == 1, ['2nd Most Common Venue']]['2nd Most Common Venue'].tolist())

{'Park': 7,
 'Yoga Studio': 4,
 'Japanese Restaurant': 2,
 'Bar': 2,
 'Coffee Shop': 7,
 'Trail': 2,
 'Italian Restaurant': 2,
 'Café': 7,
 'Bank': 2,
 'Bakery': 3,
 'Clothing Store': 2,
 'Hotel': 2,
 'Vietnamese Restaurant': 2,
 'Pet Store': 2,
 'Sandwich Place': 3,
 'Pizza Place': 2,
 'Fried Chicken Joint': 2}

In [36]:
# 3rd Most Common Venue
getDuplicatesWithCount(toronto_merged_3.loc[toronto_merged_3['Cluster Labels'] == 1, ['3rd Most Common Venue']]['3rd Most Common Venue'].tolist())

{'Coffee Shop': 10,
 'Bakery': 2,
 'Boutique': 2,
 'Café': 4,
 'Pharmacy': 3,
 'Pub': 3,
 'Deli / Bodega': 2,
 'Yoga Studio': 6,
 'Park': 4,
 'Restaurant': 7,
 'Mexican Restaurant': 2,
 'Athletics & Sports': 5,
 'Breakfast Spot': 2,
 'Bar': 3}

From the three most common venues we notice plenty of Coffee Shops, Cafés, Pizza Places, Yoga Studios and Parks as shared Most Common venues. Noticeable is the mix of several types of restaurants such as Mexican, Italian, Vietnamese and Fast Food. Compared to the other clusters, these neighbourhoods contain more food related venues. It is advised that in a further analysis this is taken into account.

The third and final cluster with only two data points contains the following venues:
1. Baseball Fields and Yoga Studio
1. Escape Room and Electronics Store
1. Eastern European Restaurant and Dumpling Restaurant
1. Drugstore and Donut Shop

This is quite similar to the first cluster in terms of uniquely found venues with the exception of the Park venue. This type of venue is not found in the cluster. Since the Park venue plays a significant role in the identity of the first cluster, these data points were most likely put into a separate cluster.

In [37]:
# The Third Cluster
toronto_merged_3.loc[toronto_merged_3['Cluster Labels'] == 2, toronto_merged_3.columns[[1] + list(range(5, toronto_merged_3.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
57,North York,2,Baseball Field,Yoga Studio,Event Space,Escape Room,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Dog Run
101,Etobicoke,2,Construction & Landscaping,Baseball Field,Yoga Studio,Dessert Shop,Escape Room,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop
